# Introduction to TensorFlow Tutorial

- Built to run on multiple CPUs or GPUs and mobile operating systems
- Written in c++, but has several wrappers in many languages

- A model is represented as a data flow graph
- The graph contains a set of nodes called operations, such as addition, multiplication, nonlinear functions
- Each operation takes as input a tensor and outputs a tensor
- Tensor: A multidimensional array of numbers, how data is represented in TensorFlow

In [1]:
# !pip install tensorflow

# Import a helper function from TensorFlow
from tensorflow.examples.tutorials.mnist import input_data
# Download the data, save it to the mnist_data folder,
# and process it so that data is in one-hot encoded format.
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

# Import TensorFlow
import tensorflow as tf

In [ ]:
# One-hot encoding: Our y variable for each sample would appear as follows:
#   1 2 3 4 5 6 7 8 9 0
# [ 1 0 0 0 0 0 0 0 0 0 ]

In [18]:
# mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0 # Not sure why Google does this

In [19]:
# x_train.shape

(60000, 28, 28)

In [20]:
# x_test.shape

(10000, 28, 28)

In [2]:
# Set parameters
learning_rate = 0.01
training_epochs = 30
batch_size = 100
display_step = 1

In [3]:
# Placeholder: A variable that we'll assign data to at a later date.
# It's never initialized and contains no data.
# Here, we pass in the type and shape of our data as parameters.
#
# TensorFlow graph input
# `None` is the batch size (the number of examples for the current batch)
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

In [4]:
# Create a model

# Weights are probabilities that affect how data flows in the graph and will continuously
# be updated so that our results get closer to the solution.
# The bias let's us shift our regression line to better fit the data.
#
# Set model weights and biases
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [5]:
# A named scope helps us organize nodes in the graph visualizer called TensorBoard.
# We'll create three scopes:
# 1) Implement softmax regression model by matrix multiplying input images X by
# weight matrix W and then adding the bias b.
#
# Create a named scope
with tf.name_scope("xW_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b)
    
# Summary operations that help us visualize the distribution of our weights and biases.
#
# Add summary operations to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [6]:
# 2) Create our cost function to help minimize error during training.
#
# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)

In [7]:
# 3) Create our optimization function that will let our model improve during training.
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [8]:
# Initializing the variables (assign their default values)
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

In [18]:
# Launch the graph
with tf.Session() as sess:
    # Initializing a session let's us execute our data flow graph
    sess.run(init)

    # Change this to a location on your computer.
    # We'll later visualize the data from this directory using TensorBoard.
    summary_writer = tf.summary.FileWriter('logs', graph=sess.graph)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Compute the total number of batches
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            # Retrieve the current batch of training samples
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # Fit our model using batch data in the gradient descent algorithm
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            
            # Compute the average loss for the current batch
            cost = sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += cost/total_batch
            
            # Write logs for each epoch
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, epoch*total_batch + i)
            
        # Display logs per display step, to ensure our model is improving during training
        if epoch % display_step == 0:
            print(f'Epoch: {(epoch + 1):04d}, loss = {avg_cost:.6f}, '
                  f'W_sum = {sess.run(tf.reduce_sum(W)):.6f}, '
                  f'b_sum = {sess.run(tf.reduce_sum(b)):.6f}')

    # Test the model by comparing model values to output values
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001, loss = 29.275465, W_sum = 0.000157, b_sum = 0.000003
Epoch: 0002, loss = 21.989802, W_sum = 0.000303, b_sum = 0.000005
Epoch: 0003, loss = 20.949330, W_sum = 0.000477, b_sum = 0.000009
Epoch: 0004, loss = 20.974767, W_sum = 0.000619, b_sum = 0.000011
Epoch: 0005, loss = 19.965329, W_sum = 0.000755, b_sum = 0.000007
Epoch: 0006, loss = 20.116556, W_sum = 0.000921, b_sum = 0.000008
Epoch: 0007, loss = 19.668701, W_sum = 0.001192, b_sum = 0.000014
Epoch: 0008, loss = 19.471331, W_sum = 0.001387, b_sum = 0.000014
Epoch: 0009, loss = 19.544566, W_sum = 0.001581, b_sum = 0.000011
Epoch: 0010, loss = 19.309294, W_sum = 0.001801, b_sum = 0.000015
Epoch: 0011, loss = 18.704587, W_sum = 0.002003, b_sum = 0.000020
Epoch: 0012, loss = 19.326377, W_sum = 0.002235, b_sum = 0.000021
Epoch: 0013, loss = 18.729991, W_sum = 0.002466, b_sum = 0.000026
Epoch: 0014, loss = 18.696948, W_sum = 0.002720, b_sum = 0.000021
Epoch: 0015, loss = 19.011172, W_sum = 0.002939, b_sum = 0.000017
Epoch: 001

In [19]:
# Run the following line to visualize the model in TensorBoard
!tensorboard --logdir='logs'

TensorBoard 1.9.0 at http://Marifels-MBP-2.home:6006 (Press CTRL+C to quit)
W0915 13:21:53.930725 Thread-2 application.py:293] path /robots.txt not found, sending 404
W0915 13:21:56.663722 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0915 13:21:56.664203 Reloader tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0915 13:22:04.970661 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0915 13:22:04.970929 Reloader tf_logging.py:120] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0915 13:22:12.926516 Reloader tf_logging.py:120] Found more than one graph event per run, or there

In [ ]:
# Events: Shows the output of cost function over time
# Histograms: Shows the variance in weights and biases over time
# Graphs: Shows the created graph and variables for the weights and biases
# (double-click for a detailed view)